In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [3]:
# Read in training data, popping off what we want to predict
train_data = pandas.read_csv('train.csv')
test_data = pandas.read_csv('test.csv')

test_pIds = test_data.get('PassengerId')

# === Clean Data ===
train_data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
medAge = train_data.Age.median()
medFare = train_data.Fare.median()
train_data['Age'].fillna(medAge, inplace=True)
train_data['Fare'].fillna(medFare, inplace=True)
train_data['Sex'].replace('male', 0, inplace=True)
train_data['Sex'].replace('female', 1, inplace=True)
train_data['Embarked'].replace('S', 0, inplace=True)
train_data['Embarked'].replace('C', 1, inplace=True)
train_data['Embarked'].replace('Q', 2, inplace=True)

# make 15% of training data validation data that isn't used for training
train_data, validation_data = train_test_split(train_data, test_size=0.15)

y_train_data = train_data.pop('Survived')
y_train = np.zeros((len(y_train_data), 2)) # two categories, survived or not survived
y_train[np.arange(len(y_train_data)), y_train_data] = 1
y_validate_data = validation_data.pop('Survived')
y_validate = np.zeros((len(y_validate_data), 2))
y_validate[np.arange(len(y_validate_data)), y_validate_data] = 1

X_train = train_data.values
X_validate = validation_data.values



# Remove anywhere a non number exists (Embarked)
y_train = y_train[~np.isnan(X_train).any(axis=1)]
X_train = X_train[~np.isnan(X_train).any(axis=1)]

test_data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
test_data['Age'].fillna(medAge, inplace=True)
test_data['Fare'].fillna(medFare, inplace=True)
test_data['Sex'].replace('male', 0, inplace=True)
test_data['Sex'].replace('female', 1, inplace=True)
test_data['Embarked'].replace('S', 0, inplace=True)
test_data['Embarked'].replace('C', 1, inplace=True)
test_data['Embarked'].replace('Q', 1, inplace=True)
X_test = test_data.values

X_test = X_test[~np.isnan(X_test).any(axis=1)]


In [4]:

num_inputs = len(X_train[0])
num_outputs = len(y_train[0]); # survived or not survived

num_hidden_outputs = 2

# Input

x = tf.placeholder(tf.float32, (None, num_inputs), name='x')
y = tf.placeholder(tf.float32, (None, num_outputs), name='y')
ec = tf.argmax(y, 1, name='ec')

# Output

output = x
logit = None
layers = 1

outputs = 32
prev_inputs = int(output.get_shape()[1])
b1 = tf.Variable(tf.zeros(outputs), name='b'+str(layers))
W1 = tf.Variable(tf.truncated_normal([prev_inputs, outputs]), name='W'+str(layers))
a1 = tf.add(tf.matmul(output, W1), b1, name='a'+str(layers))
z1 = tf.nn.softmax(a1, name='z'+str(layers))
output=z1
layers += 1

outputs = 8
prev_inputs = int(output.get_shape()[1])
b2 = tf.Variable(tf.zeros(outputs), name='b'+str(layers))
W2 = tf.Variable(tf.truncated_normal([prev_inputs, outputs]), name='W'+str(layers))
a2 = tf.add(tf.matmul(output, W2), b2, name='a'+str(layers))
z2 = tf.nn.softmax(a2, name='z'+str(layers))
output=z2
layers += 1

outputs = 2
prev_inputs = int(output.get_shape()[1])
b3 = tf.Variable(tf.zeros(outputs), name='b'+str(layers))
W3 = tf.Variable(tf.truncated_normal([prev_inputs, outputs]), name='W'+str(layers))
a3 = tf.add(tf.matmul(output, W3), b3, name='a'+str(layers))
z3 = tf.nn.softmax(a3, name='z'+str(layers))
output=z3
layers += 1


logit = a3


pc = tf.argmax(output, 1, name='pc')  # [p]redicted [c]lass label

# Loss
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logit)
loss = tf.reduce_mean(cross_entropy, name='loss')

# Solver
solver = tf.train.AdamOptimizer()
optimize = solver.minimize(loss)

# Accuracy
correct = tf.equal(ec, pc, name='correct')  # for each sample, did we get it right? 
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name='acc')

In [5]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [6]:
# Current Accuracy (before training)
print ("Accuracy:", sess.run(accuracy, {x:X_train, y:y_train}))
print ("Random chance is around 0.5 (50%)")

Accuracy: 0.382781
Random chance is around 0.5 (50%)


In [7]:
# Training cycle
training_epochs=100000
display_epoch=500

# try:
#     saver.restore(sess, './models/logreg.cpt')
# except:
#     print "No model to restore"

# val_acc = sess.run(accuracy, {x:X_validate, y:y_validate})
loss_   = sess.run(accuracy, {x:X_train, y:y_train})

for epoch in range(training_epochs):
    # Display logs per epoch step
    if (epoch) % display_epoch == 0:
        val_acc = sess.run(accuracy, {x:X_validate, y:y_validate})
        print ("Epoch:", '%04d' % (epoch), 
               "cost=", "{:.9f}".format(loss_), 
               "accuracy=","{:.9f}".format(val_acc))
    
    
    _, loss_ = sess.run([optimize, loss],
                            {x: X_train, y: y_train})
        

Epoch: 0000 cost= 0.382781446 accuracy= 0.335820884
Epoch: 0500 cost= 0.552103996 accuracy= 0.820895433
Epoch: 1000 cost= 0.441312373 accuracy= 0.843283594
Epoch: 1500 cost= 0.404596299 accuracy= 0.843283594
Epoch: 2000 cost= 0.396535039 accuracy= 0.835820913
Epoch: 2500 cost= 0.392212808 accuracy= 0.843283594
Epoch: 3000 cost= 0.383781135 accuracy= 0.828358173
Epoch: 3500 cost= 0.365747154 accuracy= 0.813432813
Epoch: 4000 cost= 0.357986093 accuracy= 0.805970132
Epoch: 4500 cost= 0.352259517 accuracy= 0.813432813
Epoch: 5000 cost= 0.347087979 accuracy= 0.820895493
Epoch: 5500 cost= 0.342618048 accuracy= 0.813432813
Epoch: 6000 cost= 0.339221656 accuracy= 0.820895493
Epoch: 6500 cost= 0.336731821 accuracy= 0.820895493
Epoch: 7000 cost= 0.334296942 accuracy= 0.813432753
Epoch: 7500 cost= 0.332490116 accuracy= 0.828358173
Epoch: 8000 cost= 0.330542535 accuracy= 0.835820854
Epoch: 8500 cost= 0.328257203 accuracy= 0.820895493
Epoch: 9000 cost= 0.322608590 accuracy= 0.820895493
Epoch: 9500 

Epoch: 77500 cost= 0.283155054 accuracy= 0.805970132
Epoch: 78000 cost= 0.282867044 accuracy= 0.813432813
Epoch: 78500 cost= 0.282840818 accuracy= 0.813432813
Epoch: 79000 cost= 0.282814980 accuracy= 0.813432813
Epoch: 79500 cost= 0.282895297 accuracy= 0.813432813
Epoch: 80000 cost= 0.282801330 accuracy= 0.813432813
Epoch: 80500 cost= 0.282776535 accuracy= 0.813432813
Epoch: 81000 cost= 0.282754481 accuracy= 0.813432813
Epoch: 81500 cost= 0.282831848 accuracy= 0.813432813
Epoch: 82000 cost= 0.282753915 accuracy= 0.813432813
Epoch: 82500 cost= 0.282720983 accuracy= 0.813432813
Epoch: 83000 cost= 0.282699496 accuracy= 0.813432813
Epoch: 83500 cost= 0.282758623 accuracy= 0.813432813
Epoch: 84000 cost= 0.282684356 accuracy= 0.813432813
Epoch: 84500 cost= 0.282663375 accuracy= 0.813432813
Epoch: 85000 cost= 0.284417063 accuracy= 0.813432813
Epoch: 85500 cost= 0.282661676 accuracy= 0.813432813
Epoch: 86000 cost= 0.282631755 accuracy= 0.813432813
Epoch: 86500 cost= 0.282613754 accuracy= 0.813

In [8]:
pred_test = sess.run([pc], {x:X_test})[0]

ret = pandas.DataFrame({'PassengerId':test_pIds, 'Survived':pred_test})
ret.to_csv('kaggle_submission_tensorflow_5a.csv', index=False)